In [10]:
%pip install langchain langgraph openai faiss-cpu langchain-community langchain-openai


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv
import os

In [24]:
load_dotenv()  # Loads from .env
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY is missing or not loaded from .env")

# Example documents/StateGrap
docs = [
    Document(page_content="LangGraph is a library for building stateful, multi-agent applications with LangChain."),
    Document(page_content="LangChain is a framework for developing applications powered by language models.")
]


embedding_model = OpenAIEmbeddings(api_key=api_key)

# Create FAISS vectorstore from documents
vectorstore = FAISS.from_documents(docs, embedding_model)

retriever = vectorstore.as_retriever()

ValueError: OPENAI_API_KEY is missing or not loaded from .env

In [22]:
# Initialize LLM
llm = ChatOpenAI(temperature=0,api_key=api_key)


memory = ConversationBufferMemory(return_messages=True)

# Rephrase question chain
rephrase_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that reformulates questions for better clarity."),
    ("user", "{input}")
])
rephrase_chain = LLMChain(llm=llm, prompt=rephrase_prompt)

# Answer generation chain
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Answer the question using the provided context."),
    ("user", "Context: {context}\n\nQuestion: {question}")
])
answer_chain = LLMChain(llm=llm, prompt=answer_prompt)


C:\Users\Rushil\AppData\Local\Temp\ipykernel_16504\140685205.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0,api_key=api_key)


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0, 'model...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
class QAState(dict):
    
    pass

def input_node(state):
    
    return {"question": state["question"]}

def rephrase_node(state):
    new_q = rephrase_chain.run(input=state["question"])
    return {"question": new_q}

def retrieve_node(state):
    docs = retriever.get_relevant_documents(state["question"])
    context = "\n".join([doc.page_content for doc in docs])
    return {"context": context}

def answer_node(state):
    answer = answer_chain.run(context=state["context"], question=state["question"])
    return {"answer": answer}

def output_node(state):
    print("\n SampleChatBot:", state["answer"])
    return state


In [ ]:
builder = StateGraph(QAState)
builder.add_node("input", input_node)
builder.add_node("rephrase", rephrase_node)
builder.add_node("retrieve", retrieve_node)
builder.add_node("answer", answer_node)
builder.add_node("output", output_node)

builder.set_entry_point("input")
builder.add_edge("input", "rephrase")
builder.add_edge("rephrase", "retrieve")
builder.add_edge("retrieve", "answer")
builder.add_edge("answer", "output")
builder.add_edge("output", END)

graph = builder.compile()


In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit","quit"]:
        print("Exiting chat. Goodbye!")
        break
    graph.invoke({"question": user_input})